In [85]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np

import random
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, Activation, Reshape
from keras.optimizers import RMSprop
from keras import backend as K
from keras.layers.convolutional import Conv1D, Conv2D
from keras import optimizers
import sys
import keras.utils
import os
sys.path.append(os.path.join(sys.path[0], '../utils'))
from load_mat_file import load_mat_file
from keras.callbacks import TensorBoard
import copy
#import matplotlib as plt

In [139]:
def create_network(input_dim, num_classes):
    '''Base network to be shared (eq. to feature extraction).
    '''
    seq = Sequential()
    seq.add(Dense(128, input_shape=(input_dim,)))
    seq.add(Activation('relu'));
    seq.add(Dropout(0.1))
    seq.add(Dense(128, activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(num_classes, activation='relu'))
    model.add(Activation('softmax'))
    return seq


def create_network_tu(max_words, num_classes):
    model = Sequential()
    model.add(Dense(512, input_shape=(max_words,)))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    return model;


def compute_accuracy(predictions, labels):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return labels[predictions.ravel() < 0.5].mean()


def create_network_eeg_conv1(input_dim, num_classes):
    seq = Sequential()
    seq.add(Reshape((1, input_dim,), input_shape=(input_dim,)))
    seq.add(Conv1D(32, 51, strides=25, padding='same', input_shape=(1, input_dim,), activation='relu'))
    seq.add(Activation('relu'));
    seq.add(Dense(units=128))
    seq.add(Activation('relu'));
    seq.add(Dense(units=num_classes))
    seq.add(Activation('softmax'));
    return seq;

In [87]:
os.getcwd()

'/home/roma/work/projects/eeg_net/src/notebooks'

In [143]:
EEG_FILENAME = 'D0000168'
CHANNELS_NUM = 21
sel_channels = slice(0, CHANNELS_NUM)
sel_time = slice(None,None,1) #quanta time
EEG = load_mat_file('../../data/eegmat_selected/' + EEG_FILENAME, 's')
X_data = EEG["eeg"][0][0]
Y_data = EEG["mrk"][0][0]
#X_data = load_mat_file('../../data/test_data2', 'X');
#Y_data = load_mat_file('../../data/test_data2', 'Y');
#Y_data= np.swapaxes(Y_data, 1,0)
X_data = np.swapaxes(X_data, 2,0)
X_data = np.swapaxes(X_data, 1,2)
print('X_data_shape: ', X_data.shape)
print('Y_data_shape: ', Y_data.shape)
OneChannel_data = X_data[:,sel_time,sel_channels];
OneChannel_data = OneChannel_data.reshape(OneChannel_data.shape[0], OneChannel_data.shape[1]*OneChannel_data.shape[2])
print('X_data_reshapedshape: ', OneChannel_data.shape)
size_of_sample = OneChannel_data.shape[1];
number_of_samples = OneChannel_data.shape[0];
print('number of samples:', number_of_samples)
print('size_of_sample:', size_of_sample)
Y_data
Y_data = Y_data - 1;
Y_data = Y_data.reshape(Y_data.shape[0])
num_classes = max(Y_data) + 1;
print('num_classes: ', num_classes)
y_= keras.utils.to_categorical(Y_data, num_classes);
x_= OneChannel_data;
test_prop = 0.1;
test_size= round(test_prop * number_of_samples, 0);
test_size = int(test_size)
train_size = number_of_samples - test_size;

X_data_shape:  (180, 275, 21)
Y_data_shape:  (180, 1)
X_data_reshapedshape:  (180, 5775)
number of samples: 180
size_of_sample: 5775
num_classes:  2


In [144]:
#random shuffle of dataset and train-test split
idx = np.random.choice(np.arange(x_.shape[0]), x_.shape[0], replace=False)
idx_train = idx[0:train_size]
idx_test = idx[train_size:]
print('train size: ', train_size)
print('test size: ', test_size)
y_train = y_[idx_train];
x_train = x_[idx_train];
y_test= y_[idx_test];
x_test= x_[idx_test];

train size:  162
test size:  18


In [ ]:
batch_size = 1;
epochs = 20;


#model = create_network_eeg_conv1(size_of_sample, num_classes)
model = create_network_tu(size_of_sample, num_classes)
sgd = optimizers.Adagrad(lr=0.0005, decay=1e-5)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

tensorboard=TensorBoard(log_dir='./logs', write_graph=True)

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.3,
                    callbacks=[tensorboard])

score = model.evaluate(x_test, y_test, batch_size=batch_size)
print(score)

Train on 113 samples, validate on 49 samples
Epoch 1/20
113/113 [==============================] - 4s 35ms/step - loss: 5.7552 - acc: 0.4159 - val_loss: 4.1987 - val_acc: 0.4286
Epoch 2/20
113/113 [==============================] - 4s 34ms/step - loss: 2.4384 - acc: 0.6637 - val_loss: 3.7649 - val_acc: 0.4490
Epoch 3/20
113/113 [==============================] - 4s 34ms/step - loss: 1.1738 - acc: 0.7788 - val_loss: 3.8245 - val_acc: 0.4490
Epoch 4/20
113/113 [==============================] - 4s 34ms/step - loss: 0.9821 - acc: 0.8142 - val_loss: 3.5938 - val_acc: 0.4286
Epoch 5/20
113/113 [==============================] - 4s 34ms/step - loss: 0.3356 - acc: 0.8850 - val_loss: 3.5912 - val_acc: 0.4694
Epoch 6/20
113/113 [==============================] - 4s 34ms/step - loss: 0.4726 - acc: 0.9027 - val_loss: 3.8077 - val_acc: 0.4082
Epoch 7/20
113/113 [==============================] - 4s 34ms/step - loss: 0.3455 - acc: 0.9027 - val_loss: 3.7438 - val_acc: 0.4694
Epoch 8/20
113/113 [====